In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [3]:
pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 90 kB 3.7 MB/s 
     |████████████████████████████████| 5.2 MB 59.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 88 kB 8.4 MB/s 
     |████████████████████████████████| 85 kB 5.4 MB/s 
     |████████████████████████████████| 662 kB 57.8 MB/s 
     |████████████████████████████████| 5.8 MB 52.1 MB/s 
     |████████████████████████████████| 1.3 MB 56.8 MB/s 
     |████████████████████████████████| 182 kB 58.5 MB/s 
     |████████████████████████████████| 7.6 MB 50.4 MB/s 
     |████████████████████████████████| 1.1 MB 58.1 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.29-cp38-cp38-linux_x86_64.whl size=2700847 sha256=81a900876c6168462cde2ad2f9698f9434fe0a2871bfd5df67d1531ad4c9f3bf
  Stored in directory: /root/.cache/

In [4]:
from bertopic import BERTopic
BERT_model_FINAL = BERTopic.load("/content/drive/MyDrive/collectionandqueries/20KDocs/qrels_BERT_model_FINAL_preprocessed")

In [19]:
from math import log, sqrt
ALPHA = 1
BETA = 0.75
GAMMA = 0.15

def generateInvertedIndex(invertedIndex, doc):
        line = doc
        text = line.strip().split()
        for word in text:
            if word not in invertedIndex.keys():
                invertedIndex[word] = 1
            else:
                invertedIndex[word] += 1

def queryFrequency(query):
    queryFreq = {}
    for term in query.split():
        if term in queryFreq.keys():
            queryFreq[term] += 1
        else:
            queryFreq[term] = 1
    return queryFreq

def calculateDocsCount(doc, docIndex):
    line = doc
    text = line.strip().split()
    for term in text:
      if term in docIndex.keys():
        docIndex[term] += 1
      else:
        docIndex[term] = 1

def findDocs(all_docs):
    relIndex = {}
    k = len(all_docs)
    for i in range(0, k):
      doc = all_docs[i]
      calculateDocsCount(doc, relIndex)
    return relIndex

def findRelDocMagnitude(docIndex):
    mag = 0
    for term in docIndex:
        mag += float(docIndex[term]**2)
        mag = float(sqrt(mag))
    return mag

def findRocchioScore(term, queryFreq, relDocMag, relIndex):
    if(term not in queryFreq.keys()):
      queryFreq[term] = 0
    if(term not in relIndex.keys()):
      relIndex[term] = 0  
    rocchioScore = ALPHA * queryFreq[term] + (BETA/relDocMag) * relIndex[term] 
    return rocchioScore


def findNewQuery(query, all_docs):
    invertedIndex = {}
    for doc in all_docs:
      generateInvertedIndex(invertedIndex, doc) 
    queryFreq = queryFrequency(query)
    relIndex = findDocs(all_docs)
    relDocMag = findRelDocMagnitude(relIndex)
    updatedQuery = {}
    newQuery = query
    for term in invertedIndex.keys():
        updatedQuery[term] = findRocchioScore(term, queryFreq, relDocMag, relIndex)
    sortedUpdatedQuery = sorted(updatedQuery.items(), key=lambda x:x[1], reverse=True)
    if len(sortedUpdatedQuery)<5:
        loopRange = len(sortedUpdatedQuery)
    else:
        loopRange = 5
    for i in range(loopRange):
        term,frequency = sortedUpdatedQuery[i]
        if term not in query:
            newQuery +=  " "
            newQuery +=  term
    return newQuery

In [7]:
import csv
import sys

csv.field_size_limit(sys.maxsize)


fr1 = open("/content/drive/MyDrive/collectionandqueries/all_query_ids.tsv", "r")
fr2 = open("/content/drive/MyDrive/collectionandqueries/all_queries.tsv", "r")
fr3 = open("/content/drive/MyDrive/collectionandqueries/all_document_ids.tsv", "r")
fr4 = open("/content/drive/MyDrive/collectionandqueries/all_documents.tsv", "r")
fr5 = open("/content/drive/MyDrive/collectionandqueries/all_labels.tsv", "r")

input_file1 = csv.reader(fr1, delimiter = "\t")
input_file2 = csv.reader(fr2, delimiter = "\t")
input_file3 = csv.reader(fr3, delimiter = "\t")
input_file4 = csv.reader(fr4, delimiter = "\t")
input_file5 = csv.reader(fr5, delimiter = "\t")

all_query_ids2 = []
all_queries2 = []
all_document_ids2 = []
all_documents2 = []
all_labels2 = []

for w in input_file1: 
    #print('w : ' + str(w[0]))
    all_query_ids2.append(w[0])

for w in input_file2: 
    all_queries2.append(w[0])

for w in input_file3: 
    all_document_ids2.append(w[0])

for w in input_file4: 
    all_documents2.append(w[0])

for w in input_file5: 
    all_labels2.append(w[0]) 

print('len(all_query_ids2) : ' + str(len(all_query_ids2)))
print('len(all_queries2) : ' + str(len(all_queries2)))
print('len(all_document_ids2) : ' + str(len(all_document_ids2)))
print('len(all_documents2) : ' + str(len(all_documents2)))
print('len(all_labels2) : ' + str(len(all_labels2)))

all_query_ids = []
all_queries = []
all_document_ids = []
all_documents = []
all_labels = []

length = len(all_query_ids2)
index = 0

for index in range(0, length): 
    #print('index : ' + str(index))
    all_query_ids.append(all_query_ids2[index])
    all_queries.append(all_queries2[index])
    all_document_ids.append(all_document_ids2[index])
    all_documents.append(all_documents2[index])
    all_labels.append(int(all_labels2[index]))
    """
    print('all_query_ids2[index] : ' + all_query_ids2[index])
    print('all_queries2[index] : ' + all_queries2[index])
    print('all_document_ids2[index] : ' + all_document_ids2[index])
    print('all_documents2[index] : ' + all_documents2[index])
    print('all_labels2[index] : ' + all_labels2[index])
    """
    
    if(int(all_labels2[index])) == 1:
      cur_query = all_queries2[index]
      print('cur_query : ' + cur_query)
      topics_res, similarity = BERT_model_FINAL.find_topics(cur_query, top_n=2) 
      for j in range(0, 2):
        all_topic_docs = BERT_model_FINAL.get_representative_docs(topics_res[j])
        newQuery = findNewQuery(cur_query, all_topic_docs)
        print('newQuery : ' + newQuery)
        all_query_ids.append(all_query_ids2[index])
        all_queries.append(newQuery)
        all_document_ids.append(all_document_ids2[index])
        all_documents.append(all_documents2[index])
        all_labels.append(int(all_labels2[index]))

print('len(all_query_ids) : ' + str(len(all_query_ids)))
print('len(all_queries) : ' + str(len(all_queries)))
print('len(all_document_ids) : ' + str(len(all_document_ids)))
print('len(all_documents) : ' + str(len(all_documents)))
print('len(all_labels) : ' + str(len(all_labels)))

len(all_query_ids2) : 8000
len(all_queries2) : 8000
len(all_document_ids2) : 8000
len(all_documents2) : 8000
len(all_labels2) : 8000
cur_query : how long will chickens lay eggs
newQuery : how long will chickens lay eggs days hatch hatching
newQuery : how long will chickens lay eggs boil hard easy
cur_query : how long will chickens lay eggs
newQuery : how long will chickens lay eggs days hatch hatching
newQuery : how long will chickens lay eggs boil hard easy
cur_query : how long will chickens lay eggs
newQuery : how long will chickens lay eggs days hatch hatching
newQuery : how long will chickens lay eggs boil hard easy
cur_query : how long will chickens lay eggs
newQuery : how long will chickens lay eggs days hatch hatching
newQuery : how long will chickens lay eggs boil hard easy
cur_query : what is the current rate for a business loan
newQuery : what is the current rate for a business loan mortgage refine credit
newQuery : what is the current rate for a business loan close fee appra

KeyboardInterrupt: ignored

In [17]:
import json

final_full_docs = []

f = open("/content/drive/MyDrive/collectionandqueries/tushita/document_data_5000.json", "r")
data = json.load(f)

for k, v in data.items():
    final_full_docs.append(v)

In [18]:
from bertopic import BERTopic
print('len(final_full_docs) : ' + str(len(final_full_docs)))

model2 = BERTopic(language='english', calculate_probabilities=True, verbose=True)
topics, probs = model2.fit_transform(final_full_docs)
model2.save("/content/drive/MyDrive/collectionandqueries/20KDocs/trec_covid_qrels_BERT_model_FINAL_preprocessed")

len(final_full_docs) : 28941


Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/905 [00:00<?, ?it/s]

2022-12-14 01:04:49,268 - BERTopic - Transformed documents to Embeddings
2022-12-14 01:05:52,375 - BERTopic - Reduced dimensionality
2022-12-14 01:09:59,064 - BERTopic - Clustered reduced embeddings
/usr/local/lib/python3.8/dist-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [20]:
from bertopic import BERTopic
TREC_COVID_BERT_model_FINAL = BERTopic.load("/content/drive/MyDrive/collectionandqueries/20KDocs/trec_covid_qrels_BERT_model_FINAL_preprocessed")

In [26]:
cur_queries = ["what is the origin of COVID-19", 
"how does the coronavirus respond to changes in the weather",
"will SARS-CoV2 infected people develop immunity? Is cross protection possible?",
"what causes death from Covid-19?",
"what drugs have been active against SARS-CoV or SARS-CoV-2 in animal studies?what types of rapid testing for Covid-19 have been developed?"]

In [24]:
cur_queries = ["which SARS-CoV-2 proteins-human proteins interactions indicate potential for drug targets. Are there approved drugs that can be repurposed based on this information?", 
               "How does the coronavirus differ from seasonal flu?",
                "is remdesivir an effective treatment for COVID-19?",
                "what evidence is there related to COVID-19 super spreaders?"]

In [27]:
for cur_query in cur_queries:
  print('cur_query : ' + cur_query)
  topics_res, similarity = TREC_COVID_BERT_model_FINAL.find_topics(cur_query, top_n=2) 
  for j in range(0, 2):
    all_topic_docs = TREC_COVID_BERT_model_FINAL.get_representative_docs(topics_res[j])
    newQuery = findNewQuery(cur_query, all_topic_docs)
    print('newQuery : ' + newQuery)

cur_query : what is the origin of COVID-19
newQuery : what is the origin of COVID-19 patient covid-19 . patients ct
newQuery : what is the origin of COVID-19 ct pregnant patients covid-19 cases
cur_query : how does the coronavirus respond to changes in the weather
newQuery : how does the coronavirus respond to changes in the weather sars-cov sars also
newQuery : how does the coronavirus respond to changes in the weather climate health et al.
cur_query : will SARS-CoV2 infected people develop immunity? Is cross protection possible?
newQuery : will SARS-CoV2 infected people develop immunity? Is cross protection possible? immunisation participants . parents focus
newQuery : will SARS-CoV2 infected people develop immunity? Is cross protection possible? sars-cov virus sars coronavirus also
cur_query : what causes death from Covid-19?
newQuery : what causes death from Covid-19? . cancer patients immune risk
newQuery : what causes death from Covid-19? patients influenza cases virus flu
cur_qu